In [ ]:
import os
print(os.getcwd());os.chdir('../')
print(os.getcwd())

In [ ]:
import sys

import numpy as np
import pandas as pd

from scipy.io import mmread

import MarcoPolo.QQscore as QQ

In [ ]:
dataset_name_all=[
'Kohinbulk_filtered',
'HumanLiver_filtered',
'Zhengmix8eq_filtered'  
]

In [ ]:
mode=2

In [ ]:
#gene_info=pd.read_csv('datasets/Homo_sapiens.gene_info.gz',sep='\t')

In [ ]:
dataset_name_all=dataset_name_all+["TabulaAorta_filtered",
"TabulaBladder_filtered",
"TabulaBrainMyeloid_filtered",
"TabulaBrainNonMyeloid_filtered",
"TabulaDiaphragm_filtered",
"TabulaFat_filtered",
"TabulaHeart_filtered",
"TabulaKidney_filtered",
"TabulaLargeIntestine_filtered",
"TabulaLimbMuscle_filtered",
"TabulaLiver_filtered",
"TabulaLung_filtered",
"TabulaMammaryGland_filtered",
"TabulaMarrow_filtered",
"TabulaPancreas_filtered",
"TabulaSkin_filtered",
"TabulaSpleen_filtered",
"TabulaThymus_filtered",
"TabulaTongue_filtered",
"TabulaTrachea_filtered"]

In [ ]:
dataset_simul_all=[]
for ncells_total in [1000,2000,5000,10000]:
    for prop in ['1e-2','5e-3','1e-3','5e-4']:
        for i in range(1,10+1):
            dataset_simul_all.append('Simul_{}_{}_{}_filtered'.format(ncells_total,prop,i))

In [ ]:
dataset_name_all=dataset_name_all+dataset_simul_all

In [ ]:
dataset_name_all

In [ ]:
for dataset_name in dataset_name_all[:]:
    print(dataset_name)
    
    path='datasets/extract/{}'.format(dataset_name)
    path
    
    exp_data=mmread('{}.data.counts.mm'.format(path)).toarray().astype(float)
    with open('{}.data.col'.format(path),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('{}.data.row'.format(path),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    assert exp_data.shape==(len(exp_data_row),len(exp_data_col))
    assert len(set(exp_data_row))==len(exp_data_row)
    assert len(set(exp_data_col))==len(exp_data_col)        

    exp_data_meta=pd.read_csv('{}.metadatacol.tsv'.format(path),sep='\t')

    cell_size_factor=pd.read_csv('{}.size_factor.tsv'.format(path),sep='\t',header=None)[0].values.astype(float)#.reshape(-1,1)

    x_data_intercept=np.array([np.ones(exp_data.shape[1])]).transpose()
    x_data_null=np.concatenate([x_data_intercept],axis=1)

    result_list,gamma_list_list,delta_log_list_list,beta_list_list=QQ.read_QQscore(path,[1,mode])

    gamma_list=gamma_list_list[-1]    

    gamma_argmax_list=QQ.gamma_list_exp_data_to_gamma_argmax_list(gamma_list,exp_data)#gamma_argmax_list=QQ.gamma_list_to_gamma_argmax_list(gamma_list)
    gamma_argmax_list,gamma_argmax_list.shape    
       
    
 



    """


    with open('datasets/extract/{}.data.col'.format(dataset_name),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('datasets/extract/{}.data.row'.format(dataset_name),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    allscore=allscore_munge


    crit1=allscore['QQdiff_rank'].copy()

    #(allscore[['PCvariance_rank','votingscore_rank']].min(axis=1)<(97/100*len(exp_data_row)))
    crit2=allscore['votingscore_rank'].copy()

    #(allscore[['PCvariance_rank','votingscore_rank']].min(axis=1)<(97/100*len(exp_data_row)))

    crit3=allscore['QQratio_rank'].copy()


    crit4=allscore['mean_0_all_rank'].copy()

    
    crit5=allscore['PCstd2_rank'].copy()    
    
    
    #(allscore[['votingscore_rank','PCvariance_rank']].min(axis=1)<(97/100*len(exp_data_row)))
    MarcoPolo_score=pd.concat([crit1,crit2,crit3,crit4,crit5],axis=1).sum(axis=1)    
    
    MarcoPolo_score[~(
            (allscore['minorsize']>int(10))&
            (allscore['minorsize']<int(70/100*len(exp_data_col)))        
            )]=len(allscore)  
    


    allscore['MarcoPolo']=MarcoPolo_score
    allscore['MarcoPolo_rank']=pd.Series(np.arange(allscore.shape[0]),index=allscore['MarcoPolo'].sort_values(ascending=True).index).loc[allscore.index]

    allscore_munge=allscore 


    allscore_munge['Symbol']=np.nan
    allscore_munge['Gene ID']=exp_data_row

    allscore_munge.to_csv('{}.MarcoPolo.{}.rank.munge.tsv'.format(path,mode),sep='\t')
    
    





    """





    
    marker_matrix=pd.DataFrame(index=exp_data_row, columns=exp_data_meta['phenoid'].unique())

    marker_matrix.head()   
    
    phenoid_count=np.array([np.sum((exp_data_meta['phenoid']==phenoid)) for phenoid in marker_matrix.columns])

    for idx,row in marker_matrix.iterrows():
        #print(exp_data_row.index(idx))
        exp_data_select=exp_data[exp_data_row.index(idx),:]
        gamma_argmax_select=gamma_argmax_list[exp_data_row.index(idx),:]
        phenoid_on=np.array([np.sum(gamma_argmax_select[(exp_data_meta['phenoid']==phenoid).values]==0) for phenoid in row.index])

        marker_matrix.loc[idx]=phenoid_on/phenoid_count    
        
    marker_matrix.to_csv('datasets/extract/{}.typefinding.MarcoPolo.tsv'.format(dataset_name),sep='\t')

In [ ]:
allscore_munge.set_index('Gene name').loc['MALAT1']#ENSG00000168542

In [ ]:
allscore_munge.iloc[:,:20]

In [ ]:
pd.concat([crit1,crit2,crit4,crit5],axis=1).

In [ ]:
ENSG00000280623

In [ ]:
allscore_munge.set_index('Gene ID').loc['ENSG00000168542']#ENSG00000168542

In [ ]:
allscore_munge.set_index('Gene ID').loc['ENSG00000280623']

In [ ]:
ENSG00000019485

In [ ]:
allscore_munge.sort_values('PCstd2_rank').iloc[:50]

In [ ]:
allscore_munge.sort_values('MarcoPolo_rank').iloc[:50]

In [ ]:
allscore_munge.iloc[exp_data_row.index('ENSG00000186272')]

In [ ]:
allscore_munge.iloc[exp_data_row.index('ENSG00000071282')]

In [ ]:
pd.set_option("display.max_columns", 100)

In [ ]:
allscore_munge.sort_values("MarcoPolo_rank")

In [ ]:
gamma_argmax_list

In [ ]:
for dataset_name in dataset_name_all:
    print(dataset_name)
    path='datasets/extract/{}'.format(dataset_name)
    path
    
    exp_data=mmread('{}.data.counts.mm'.format(path)).toarray().astype(float)
    with open('{}.data.col'.format(path),'r') as f: exp_data_col=[i.strip().strip('"') for i in f.read().split()]
    with open('{}.data.row'.format(path),'r') as f: exp_data_row=[i.strip().strip('"') for i in f.read().split()]
    assert exp_data.shape==(len(exp_data_row),len(exp_data_col))
    assert len(set(exp_data_row))==len(exp_data_row)
    assert len(set(exp_data_col))==len(exp_data_col)        

    exp_data_meta=pd.read_csv('{}.metadatacol.tsv'.format(path),sep='\t')

    cell_size_factor=pd.read_csv('{}.size_factor.tsv'.format(path),sep='\t',header=None)[0].values.astype(float)#.reshape(-1,1)

    x_data_intercept=np.array([np.ones(exp_data.shape[1])]).transpose()
    x_data_null=np.concatenate([x_data_intercept],axis=1)

    #gamma_argmax_list=(exp_data<=1).astype(int)
    #(exp_data<=1).astype(int)
    gamma_argmax_list=(exp_data<=np.median(exp_data,axis=1).reshape(-1,1)).astype(int)

    #gamma_list=gamma_list_list[-1]    

    #gamma_argmax_list=QQ.gamma_list_exp_data_to_gamma_argmax_list(gamma_list,exp_data)#gamma_argmax_list=QQ.gamma_list_to_gamma_argmax_list(gamma_list)
    #gamma_argmax_list,gamma_argmax_list.shape    
       
    
    
    
    #allscore=pd.read_csv('{}.MarcoPolo.{}.rank.tsv'.format(path,mode),index_col=0,sep='\t')
    allscore=pd.read_csv('datasets/extract/{}.hvg.ht.pca.tsv'.format(dataset_name),sep='\t',index_col=0)
    allscore_munge=allscore.copy()
 
    
    marker_matrix=pd.DataFrame(index=exp_data_row, columns=exp_data_meta['phenoid'].unique())

    marker_matrix.head()   
    
    phenoid_count=np.array([np.sum((exp_data_meta['phenoid']==phenoid)) for phenoid in marker_matrix.columns])

    for idx,row in marker_matrix.iterrows():
        exp_data_select=exp_data[exp_data_row.index(idx),:]
        gamma_argmax_select=gamma_argmax_list[exp_data_row.index(idx),:]
        phenoid_on=np.array([np.sum(gamma_argmax_select[(exp_data_meta['phenoid']==phenoid).values]==0) for phenoid in row.index])

        marker_matrix.loc[idx]=phenoid_on/phenoid_count    
        
    marker_matrix.to_csv('datasets/extract/{}.typefinding.ht.pca.tsv'.format(dataset_name),sep='\t')
    

In [ ]:
gamma_argmax_list.shape

In [ ]:
len(exp_data_row)